In [2]:
import sys
sys.path.append('../../../src/')

import os
from typing import Dict
from os import PathLike
from pathlib import Path

import data_io

from harp.reader import create_reader

from utils import parse, processing, plotting_utils as plotting, AddExtraColumns

# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from tkinter import font
from matplotlib.gridspec import GridSpec

from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator
import seaborn as sns
import pandas as pd
import numpy as np
import datetime

def format_func(value, tick_number):
    return f"{value:.0f}"

from numpy.typing import ArrayLike
from typing import Literal, Tuple

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [4]:
cumulative_df = pd.DataFrame(columns=['mouse', 'session', 'total_sites', 'rewarded_stops', 'unrewarded_stops', 'water', 'distance', 'session_n'])
cumulative_df_trials = pd.DataFrame()
# for mouse in ["690164","690165","690167","699894","699895","699899", "694569"]:
for mouse in ["716455"]:

    base_path = 'Z:/scratch/vr-foraging/data/'

    session_n = 0
    current_session = ''
    for file_name in os.listdir(os.path.join(base_path, mouse)):

        session_path = os.path.join(base_path, mouse, file_name)
        session = file_name[:8]
        if current_session != session:
            session_n+=1
            current_session = session
        session_path = Path(session_path)

        try:
            data = parse.load_session_data(session_path)
        except:
            print('This session is missing data: ', session)
            continue

        print(session)
        try:
            data = parse.load_session_data(session_path)
        except:
            print('Error in loading')
            continue
                
        try:
            data['harp_olfactometer'].streams.OdorValveState.load_from_file()
            data['harp_olfactometer'].streams.EndValveState.load_from_file()
        except:
            pass
        
        try:
            data['harp_behavior'].streams.OutputSet.load_from_file()
            data['harp_behavior'].streams.OutputClear.load_from_file()
            data['config'].streams['TaskLogic'].load_from_file()
        except:
            continue
        
        try:
            reward_sites, active_site, encoder_data, config = analysis.parse_data(data)
        except:
            print('Error parsing data')
            continue
            
        
        collected_df = reward_sites.loc[reward_sites['has_choice']==True].groupby(['collected','odor_label'])['reward_delivered'].count().reset_index()

        unrewarded_stops = collected_df.loc[collected_df.collected==0]['reward_delivered'].sum()
        rewarded_stops = collected_df.loc[collected_df.collected==1]['reward_delivered'].sum()
        water_collected = reward_sites.loc[(reward_sites['collected']==1)]['reward_delivered'].sum()
        total_stops = reward_sites.loc[(reward_sites['has_choice']==True)]['reward_available'].count()

        stopped_df = reward_sites.loc[(reward_sites['has_choice']==True)].groupby(['collected','odor_label'])[['reward_delivered']].sum().reset_index()
        
        print('Total sites: ' ,len(reward_sites), ' | ', 'Total rewarded stops: ',rewarded_stops, '(',  np.round((rewarded_stops/total_stops)*100,2),'%) | ', 
            'Total unrewarded stops: ',unrewarded_stops,'(',  np.round((unrewarded_stops/total_stops)*100,2),'%) | ','Water consumed: ', water_collected, 'ul',)
        if 'startPosition' in active_site.columns:
            print('Total travelled m: ', np.round(active_site.startPosition.max()/100,2))
        else:
            print('Total travelled m: ', np.round(active_site.start_position.max(),2))
            
        for odor_label in stopped_df.loc[(stopped_df.collected==1)].odor_label.unique():
            print(odor_label, ':', stopped_df.loc[(stopped_df.odor_label == odor_label)&(stopped_df.collected==1), 'reward_delivered'].iloc[0], 'ul')

        if 'startPosition' in active_site.columns:
            stop_duration = np.round(active_site.startPosition.max()/100,2)
        else:
            stop_duration = np.round(active_site.start_position.max(),2)
        print('Total travelled m: ', np.round(active_site.start_position.max(),2))
        
        cumulative_df_trials['mouse'] = mouse
        cumulative_df_trials['session'] = session
        cumulative_df_trials = pd.concat([cumulative_df_trials, reward_sites], ignore_index=True)

        # if session not in cumulative_df.session.unique() and :
        new_row = {'mouse':mouse, 'session':session, 'total_sites':len(reward_sites), 'rewarded_stops':rewarded_stops, 'unrewarded_stops':unrewarded_stops, 
                    'water':water_collected, 'distance':stop_duration, 'session_n':session_n}
        cumulative_df.loc[len(cumulative_df)] = new_row
        
        
        break


20240409
20240410
20240411
20240412
20240413
20240415
20240416
20240417
20240418
20240419
20240422
20240423
20240424
20240425
20240426
20240429
20240430
20240501
20240502
20240503
20240506
20240507
20240508
20240509
20240510
20240513
20240514
20240515
20240516
20240520
20240521
20240522


In [5]:
date = datetime.date.today()
date_string = "4/22/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

In [6]:
mouse = '715866'

In [7]:
session_found = False

directory = os.path.join(base_path, mouse)
files = os.listdir(os.path.join(base_path, mouse))

sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)

for file_name in sorted_files:
    
    if session_found == True:
        break
    
    print(file_name)
    session_path = os.path.join(base_path, mouse, file_name)
    session = file_name[:8]
    session_path = Path(session_path)
    
    if datetime.date.fromtimestamp(os.path.getctime(session_path)) != date:
        continue
    else:
        print('correct date found')
        session_found = True
    

    data = analysis.load_session_data(session_path)
    

    data['harp_olfactometer'].streams.OdorValveState.load_from_file()
    data['harp_olfactometer'].streams.EndValveState.load_from_file()
    data['software_events'].streams.ActiveSite.load_from_file()
    data['software_events'].streams.ChoiceFeedback.load_from_file()
    
    data['harp_behavior'].streams.OutputSet.load_from_file()
    data['harp_behavior'].streams.OutputClear.load_from_file()
    
    reward_sites, active_site, encoder_data, config = analysis.parse_data(data)

20240522T094504
20240521T105937
20240520T111557
20240516T095823
20240515T100348
20240514T103233
20240513T104111
20240510T100828
20240509T094946
20240508T104947
20240507T105659
20240506T100408
20240503T094055
20240502T094324
20240501T095811
20240430T102108
20240429T103414
20240426T101144
20240425T102305
20240424T102938
20240423T100551
20240422T110031
correct date found
0.38748955726623535


In [8]:
all_epochs = pd.concat([reward_sites, active_site.loc[active_site.label != 'RewardSite']])
all_epochs.sort_index(inplace=True)

In [9]:
active_patch = -1
total_sites = -1
for i, row in all_epochs.iterrows():
    if row['label'] == 'InterPatch':
        active_patch += 1
    if row['label'] == 'InterSite':
        total_sites += 1
    all_epochs.at[i, 'active_patch'] = active_patch
    all_epochs.at[i, 'total_sites'] = total_sites
all_epochs['total_sites'] = np.where(all_epochs['total_sites'] == -1, 0, all_epochs['total_sites'])

In [10]:
all_epochs.head(3)

,Seconds,label,start_position,length,reward_available,active_patch,visit_number,has_choice,reward_delivered,past_no_reward_count,...,depleted,collected,last_visit,last_site,reward_probability,success_number,patch_success_number,patch_no_reward_number,skipped_count,reward_amount
0.0,1.704920e+06,RewardSite,262.242267,60.0,40.0,-1.0,0.0,True,5.0,0.0,...,0.0,1.0,0.0,0.0,0.900000,0.0,1.0,0.0,0.0,5.0
1.0,1.704925e+06,RewardSite,373.579105,60.0,35.0,-1.0,1.0,True,5.0,0.0,...,0.0,1.0,0.0,0.0,0.877781,1.0,2.0,0.0,0.0,5.0
2.0,1.704930e+06,RewardSite,502.199200,60.0,30.0,-1.0,2.0,True,5.0,0.0,...,0.0,1.0,0.0,0.0,0.856111,2.0,3.0,0.0,0.0,5.0


In [12]:
reward_sites

,Seconds,label,start_position,length,reward_available,active_patch,visit_number,has_choice,reward_delivered,past_no_reward_count,...,depleted,collected,last_visit,last_site,reward_probability,success_number,patch_success_number,patch_no_reward_number,skipped_count,reward_amount
0,1.704920e+06,RewardSite,262.242267,60.0,40.0,0.0,0.0,True,5.0,0.0,...,0,1,0,0,0.900000,0.0,1.0,0.0,0.0,5.0
1,1.704925e+06,RewardSite,373.579105,60.0,35.0,0.0,1.0,True,5.0,0.0,...,0,1,0,0,0.877781,1.0,2.0,0.0,0.0,5.0
2,1.704930e+06,RewardSite,502.199200,60.0,30.0,0.0,2.0,True,5.0,0.0,...,0,1,0,0,0.856111,2.0,3.0,0.0,0.0,5.0
3,1.704933e+06,RewardSite,613.691244,60.0,25.0,0.0,3.0,True,5.0,0.0,...,0,1,0,0,0.834976,3.0,4.0,0.0,0.0,5.0
4,1.704937e+06,RewardSite,737.782668,60.0,20.0,0.0,4.0,True,5.0,0.0,...,0,1,0,0,0.814362,4.0,5.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,1.708404e+06,RewardSite,115262.206980,60.0,40.0,264.0,0.0,False,0.0,0.0,...,0,0,0,1,0.900000,0.0,0.0,0.0,11.0,5.0
499,1.708410e+06,RewardSite,115545.861759,60.0,40.0,265.0,0.0,False,0.0,0.0,...,0,0,0,1,0.900000,0.0,0.0,0.0,12.0,5.0
500,1.708415e+06,RewardSite,115810.527933,60.0,40.0,266.0,0.0,False,0.0,0.0,...,0,0,0,1,0.900000,0.0,0.0,0.0,13.0,5.0
501,1.708421e+06,RewardSite,116123.230677,60.0,40.0,267.0,0.0,True,5.0,0.0,...,0,1,0,0,0.900000,0.0,1.0,0.0,0.0,5.0


In [105]:
cumulative_df = pd.DataFrame(columns=['odor_0'])

In [71]:
data['config'].streams['tasklogic_input'].load_from_file()

In [124]:
cumulative_df.at[1, 'odor_' + str(i)] = data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]['label']

In [119]:
i = 0
data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]
cumulative_df['odor_' + str(i)] = data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]['label']

In [128]:
data['config'].streams['tasklogic_input'].data['environment_statistics']['patches'][i]

{'label': 'Amyl Acetate',
 'state_index': 0,
 'odor_specification': {'index': 0, 'concentration': 1.0},
 'reward_specification': {'operant_logic': {'is_operant': False,
   'stop_duration': 0.0,
   'time_to_collect_reward': 100000.0,
   'grace_distance_threshold': 10.0},
  'delay': {'family': 'Normal',
   'distribution_parameters': {'family': 'Normal', 'mean': 0.0, 'std': 0.15},
   'truncation_parameters': {'is_truncated': True, 'min': 0.0, 'max': 0.75},
   'scaling_parameters': {'scale': 1.0, 'offset': 0.0}},
  'reward_function': {'amount': {'function_type': 'ConstantFunction',
    'value': 5.0},
   'probability': {'function_type': 'PowerFunction',
    'mininum': 0.0,
    'maximum': 0.9,
    'a': 0.9,
    'b': 2.718,
    'c': -0.015,
    'd': 0.0},
   'available': {'function_type': 'LinearFunction',
    'mininum': 0.0,
    'maximum': 9999.0,
    'a': -5.0,
    'b': 40.0},
   'depletion_rule': 'OnReward'}},
 'virtual_site_generation': {'inter_site': {'render_specification': {'contrast':